In [1]:
import pickle
from pprint import pprint
import psycopg2

In [3]:
dag_ids = ['FinSum_OpenAI']
READ_COST_TEMPLATE = "./include/dag_timings/{}_timing_logs.pkl"
TOTAL_COST_TEMPLATE = "./include/dag_timings/{}_task_durations.pkl"

read_costs = {}
total_costs = {}
for dag_id in dag_ids:
    read_cost_file = READ_COST_TEMPLATE.format(dag_id)
    total_cost_file = TOTAL_COST_TEMPLATE.format(dag_id)
    with open(read_cost_file, "rb") as f:
        loaded_data = pickle.load(f)
        read_costs[dag_id] = loaded_data
    with open(total_cost_file, "rb") as f:
        loaded_data = pickle.load(f)
        total_costs[dag_id] = loaded_data

print(f"Files: {read_cost_file}, {total_cost_file}")
print('READ COSTS:')
pprint(read_costs)
print('TOTAL COSTS:')
pprint(total_costs)


Files: ./include/dag_timings/s3_upload_copy_timing_logs.pkl, ./include/dag_timings/s3_upload_copy_task_durations.pkl
READ COSTS:
{'s3_upload_copy': {}}
TOTAL COSTS:
{'s3_upload_copy': {'copy_your_name_here_file_0': 2.9717460000000004,
                    'copy_your_name_here_file_1': 2.962987,
                    'copy_your_name_here_file_2': 4.054346,
                    'copy_your_name_here_file_3': 2.934907,
                    'copy_your_name_here_file_4': 2.936482,
                    'copy_your_name_here_file_5': 4.009849999999999,
                    'copy_your_name_here_file_6': 2.957812,
                    'copy_your_name_here_file_7': 2.94966,
                    'copy_your_name_here_file_8': 2.9590170000000002,
                    'copy_your_name_here_file_9': 2.9354690000000003,
                    'generate_file_your_name_here_0': 3.4654030000000002,
                    'generate_file_your_name_here_1': 3.670889,
                    'generate_file_your_name_here_2': 3.528

In [2]:
dag_id = 'example'
POSTGRES_CONFIG = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # will be updated dynamically to point to container IP
    'port': 5432,
}

In [14]:
conn = psycopg2.connect(**POSTGRES_CONFIG)

dag_id = 'FinSum_OpenAI'
query = """
        SELECT task_id, AVG(duration) as total_duration
        FROM task_instance
        WHERE dag_id = %s AND state = 'success'
        GROUP BY task_id;
    """
with conn.cursor() as cur:
    cur.execute(query, (dag_id,))
    results = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]

conn.close()
results

[('split', 2.331185),
 ('vectorize_summaries', 0.06823270000000001),
 ('summarize_openai', 452.12073166666664),
 ('extract', 22.02540433333333),
 ('vectorize_chunks', 168.19104233333334)]

In [11]:
conn = psycopg2.connect(**POSTGRES_CONFIG)
with conn.cursor() as cur:
    cur.execute("SELECT state FROM dag_run WHERE dag_id = %s ORDER BY execution_date DESC LIMIT 1", ('blah',))
    state = cur.fetchone()
conn.close()
print(state[0])

TypeError: 'NoneType' object is not subscriptable